In [34]:
import pandas as pd
import endpoints
from datetime import datetime, timedelta
# from config import SITE_EMAIL, SITE_PASSWORD
import time
import numpy as np
import requests
import json
from urllib.parse import quote
import os

In [3]:
today = datetime.today()
yesterday = today - timedelta(days=1)
today_str = today.strftime('%Y-%m-%d')
yesterday_str = yesterday.strftime('%Y-%m-%d')

In [4]:
SITE_EMAIL = os.getenv("SITE_EMAIL")
SITE_PASSWORD = os.getenv("SITE_PASSWORD")

In [35]:
df = pd.read_csv('/Users/vnaumq/code/projects/RoadMap/mini-dwh-sales-pipeline/dags/info_30_days.csv')

In [44]:
df[['l3_id']].drop_duplicates().count()

l3_id    50
dtype: int64

In [18]:
def get_info_30_days(yesterday_str: str, l3_id: int, today_str: str, session: requests.Session, cookies_dict: dict):

    # Формируем JSON query
    query_params = {
        "start": 0,
        "length": 0,
        "orderBy": "ordersSum",
        "orderDirection": "desc",
        "checkDate": yesterday_str,
        "periodDays": 1,
        "trendType": "day",
        "filters": {
            "showFavoritesOnly": {"value": False}
        }
    }

    encoded_query = quote(json.dumps(query_params))

    # Generate dynamic dns-cache
    response = requests.post(f'https://eggheads.solutions/analytics/wbCategory/buildCache/{l3_id}',cookies=cookies_dict)
    time.sleep(1)
    url = f"https://eggheads.solutions/analytics/wbCategory/getBrandsList/{l3_id}.json?query={encoded_query}&dns-cache={today_str}_09-1"
    response = session.get(url, cookies=cookies_dict)

    if response.status_code == 200:
        data = response.json()

        return data['totals']
    else:
        print(response.text)
        print(url)

        return data['totals']

In [21]:
df = pd.read_csv('/Users/vnaumq/code/projects/RoadMap/mini-dwh-sales-pipeline/dags/l1_l2_data.csv')

In [24]:
df[df['l1_name'] == 'Сделано в России']

l1_id      58
l1_name    58
l2_id      58
l2_name    58
dtype: int64

In [26]:
df[df['l1_name'] == 'Акции'].count()

l1_id      71
l1_name    71
l2_id      71
l2_name    71
dtype: int64

In [6]:
today = datetime.today()
yesterday = today - timedelta(days=1)
today_str = today.strftime('%Y-%m-%d')
yesterday_str = yesterday.strftime('%Y-%m-%d')


cookies_dict = endpoints.get_cookies_local(SITE_EMAIL, SITE_PASSWORD)
session = endpoints.get_session()

In [ ]:
7079448242063961252

In [9]:
l3_id = 7079448242063961252

In [19]:
data = get_info_30_days(yesterday_str, l3_id, today_str, session, cookies_dict)

In [20]:
data

In [16]:
l2_id = 13228587801543660162
data = endpoints.get_l3(yesterday_str, l2_id, today_str, session, cookies_dict)
df_temp = pd.json_normalize(data['data'])


{"status":"ok","data":[{"id":"7079448242063961252","url":"https:\/\/www.wildberries.ru\/catalog\/aksessuary\/avtotovary\/avtoaksessuary-i-dopolnitelnoe-oborudovanie\/offroad","path":"Автотовары > OFFroad","name":"OFFroad","totalProducts":31626,"rating":15269.28,"orders":164,"ordersSum":320208,"avgOrdersSumForOrderedCard":580,"avgOrdersSumForCard":10,"ordersPeriodSum":13137401,"loosesPeriodSum":5081861,"clientProductsCount":0,"clientProductsRating":0,"orderedCardsCount":552,"orderedCardsCountPercent":1.75,"groupAOrderedCardsCount":71,"groupAOrderedCardsCountPercent":0.22,"groupAAvgOrderSum":3195,"groupAAvgTotalOrdersSum":130130,"loosesPeriodPercent":38.68,"isFavorite":false},{"id":"12413804913918531246","url":"https:\/\/www.wildberries.ru\/catalog\/aksessuary\/avtotovary\/avtoaksessuary-i-dopolnitelnoe-oborudovanie\/offroad?xsubject=2392","path":"Автотовары > OFFroad > Антипробуксовочное приспособление","name":"Антипробуксовочное приспособление","totalProducts":2622,"rating":2856.49,"or

In [30]:
df = df[~df['l1_name'].isin(['Сделано в России', 'Акции'])]


In [33]:
df['l2_id'].drop_duplicates()

0      13228587801543660162
1       5112727546152373259
2       3279925191302616592
3       9104072363897974717
4       9918326784635311320
               ...         
533     7661203726409346412
534    12112583902585777490
535    15914763253791550990
536    16947264205671234706
537     2352348941112735609
Name: l2_id, Length: 407, dtype: object

In [15]:
df_temp[df_temp['orders'] > df_temp['orders'].mean()]

,id,url,path,name,totalProducts,rating,orders,ordersSum,avgOrdersSumForOrderedCard,avgOrdersSumForCard,...,clientProductsCount,clientProductsRating,orderedCardsCount,orderedCardsCountPercent,groupAOrderedCardsCount,groupAOrderedCardsCountPercent,groupAAvgOrderSum,groupAAvgTotalOrdersSum,loosesPeriodPercent,isFavorite
0,7079448242063961252,https://www.wildberries.ru/catalog/aksessuary/...,Автотовары > OFFroad,OFFroad,31626,15269.28,164,320208,580,10,...,0,0,552,1.75,71,0.22,3195,130130,38.68,False
1,12413804913918531246,https://www.wildberries.ru/catalog/aksessuary/...,Автотовары > OFFroad > Антипробуксовочное прис...,Антипробуксовочное приспособление,2622,2856.49,24,25533,184,10,...,0,0,139,5.30,38,1.45,2225,30272,82.79,False
6,14155676993144649152,https://www.wildberries.ru/catalog/aksessuary/...,Автотовары > OFFroad > Главная пара,Главная пара,1191,1297.97,29,89801,1871,75,...,0,0,48,4.03,20,1.68,4004,24622,56.21,False
7,11469048034203598223,https://www.wildberries.ru/catalog/aksessuary/...,Автотовары > OFFroad > Канистра экспедиционная,Канистра экспедиционная,1372,1491.73,17,6394,128,5,...,0,0,50,3.64,11,0.80,390,25871,29.09,False
12,4330116137579132127,https://www.wildberries.ru/catalog/aksessuary/...,Автотовары > OFFroad > Лебедка автомобильная,Лебедка автомобильная,2229,2432.93,20,86208,1039,39,...,0,0,83,3.72,5,0.22,5805,1210848,14.65,False
19,11934597535938862783,https://www.wildberries.ru/catalog/aksessuary/...,Автотовары > OFFroad > Расширитель колесной арки,Расширитель колесной арки,810,882.73,17,31739,369,39,...,0,0,86,10.62,40,4.94,2547,7449,220.75,False
20,10616523740629999176,https://www.wildberries.ru/catalog/aksessuary/...,Автотовары > OFFroad > Фара дополнительного света,Фара дополнительного света,2890,3159.38,55,40706,187,14,...,0,0,218,7.54,67,2.31,1103,13282,42.51,False


In [11]:
data

{'brandId': None,
 'lastRemains': None,
 'reviewsCount': None,
 'name': 'Автотовары > OFFroad',
 'isFavorite': False,
 'orders': 6075,
 'ordersSum': 15315308,
 'loosesPercent': 57.33,
 'avgSum': 2521,
 'loosesSum': 8780892,
 'trend': [{'date': '2025-08-23T00:00:00+00:00',
   'ordersSum': 450918,
   'orders': 184},
  {'date': '2025-08-24T00:00:00+00:00', 'ordersSum': 396691, 'orders': 156},
  {'date': '2025-08-25T00:00:00+00:00', 'ordersSum': 422077, 'orders': 155},
  {'date': '2025-08-26T00:00:00+00:00', 'ordersSum': 401658, 'orders': 155},
  {'date': '2025-08-27T00:00:00+00:00', 'ordersSum': 540695, 'orders': 211},
  {'date': '2025-08-28T00:00:00+00:00', 'ordersSum': 685360, 'orders': 223},
  {'date': '2025-08-29T00:00:00+00:00', 'ordersSum': 552036, 'orders': 186},
  {'date': '2025-08-30T00:00:00+00:00', 'ordersSum': 515483, 'orders': 216},
  {'date': '2025-08-31T00:00:00+00:00', 'ordersSum': 491734, 'orders': 215},
  {'date': '2025-09-01T00:00:00+00:00', 'ordersSum': 424013, 'orders

In [45]:
data

{'status': 'ok',
 'items': [],
 'totals': None,
 'userBrands': [],
 'filters': {'orders': {'type': 'range', 'min': None, 'max': None},
  'ordersSum': {'type': 'range', 'min': None, 'max': None},
  'loosesPercent': {'type': 'range', 'min': None, 'max': None},
  'avgSum': {'type': 'range', 'min': None, 'max': None},
  'loosesSum': {'type': 'range', 'min': None, 'max': None},
  'name': {'type': 'text', 'value': None},
  'orderedCardsSum': {'type': 'range', 'min': None, 'max': None},
  'productsCount': {'type': 'range', 'min': None, 'max': None},
  'newProductsCount': {'type': 'range', 'min': None, 'max': None},
  'sharePercent': {'type': 'range', 'min': None, 'max': None},
  'lastRemains': {'type': 'range', 'min': None, 'max': None}},
 'request': {'filters': {'showFavoritesOnly': {'type': 'boolean',
    'value': False}},
  'checkDate': '2025-09-21T00:00:00+00:00',
  'periodDays': 30,
  'trendType': 'day',
  'draw': 0,
  'start': 0,
  'length': 0,
  'orderBy': 'ordersSum',
  'orderDirectio

In [36]:
data

In [ ]:
page_rul =

In [27]:
from bs4 import BeautifulSoup

response = session.get('https://eggheads.solutions/fe/wbAnalytics/category/14155676993144649152?dateTo=2025-09-21&collection=brands&period=30&trendType=day')
soup = BeautifulSoup(response.text, 'html.parser')
csrf_token = soup.find('input', {'name': 'csrf_token'})  # Пример, зависит от структуры страницы


In [28]:
csrf_token

In [23]:
data

{'brandId': None,
 'lastRemains': None,
 'reviewsCount': None,
 'name': 'Автотовары > OFFroad > Антипробуксовочное приспособление',
 'isFavorite': False,
 'orders': 787,
 'ordersSum': 1502613,
 'loosesPercent': 89.07,
 'avgSum': 1909,
 'loosesSum': 1338419,
 'trend': [{'date': '2025-08-23T00:00:00+00:00',
   'ordersSum': 34898,
   'orders': 15},
  {'date': '2025-08-24T00:00:00+00:00', 'ordersSum': 49629, 'orders': 14},
  {'date': '2025-08-25T00:00:00+00:00', 'ordersSum': 36870, 'orders': 25},
  {'date': '2025-08-26T00:00:00+00:00', 'ordersSum': 53605, 'orders': 25},
  {'date': '2025-08-27T00:00:00+00:00', 'ordersSum': 63918, 'orders': 28},
  {'date': '2025-08-28T00:00:00+00:00', 'ordersSum': 51036, 'orders': 22},
  {'date': '2025-08-29T00:00:00+00:00', 'ordersSum': 32081, 'orders': 14},
  {'date': '2025-08-30T00:00:00+00:00', 'ordersSum': 37458, 'orders': 30},
  {'date': '2025-08-31T00:00:00+00:00', 'ordersSum': 36263, 'orders': 19},
  {'date': '2025-09-01T00:00:00+00:00', 'ordersSum':

In [108]:
data

{'brandId': None,
 'lastRemains': None,
 'reviewsCount': None,
 'name': 'Книги > Политика и право',
 'isFavorite': False,
 'orders': 18699,
 'ordersSum': 11536991,
 'loosesPercent': 9.82,
 'avgSum': 617,
 'loosesSum': 1133109,
 'trend': [{'date': '2025-08-23T00:00:00+00:00',
   'ordersSum': 288076,
   'orders': 385},
  {'date': '2025-08-24T00:00:00+00:00', 'ordersSum': 307023, 'orders': 409},
  {'date': '2025-08-25T00:00:00+00:00', 'ordersSum': 282795, 'orders': 367},
  {'date': '2025-08-26T00:00:00+00:00', 'ordersSum': 328206, 'orders': 426},
  {'date': '2025-08-27T00:00:00+00:00', 'ordersSum': 392668, 'orders': 554},
  {'date': '2025-08-28T00:00:00+00:00', 'ordersSum': 457637, 'orders': 609},
  {'date': '2025-08-29T00:00:00+00:00', 'ordersSum': 372062, 'orders': 512},
  {'date': '2025-08-30T00:00:00+00:00', 'ordersSum': 445521, 'orders': 654},
  {'date': '2025-08-31T00:00:00+00:00', 'ordersSum': 512445, 'orders': 784},
  {'date': '2025-09-01T00:00:00+00:00', 'ordersSum': 424278, 'ord

In [120]:
df = pd.json_normalize(data)

In [121]:
df['l3_id'] = l3_id

In [122]:
df = df[['l3_id', 'name', 'trend']]

In [123]:
df.rename(columns={'name': 'l3_name'}, inplace=True)

In [138]:
df = pd.json_normalize(data)

df = df[['name', 'trend']]
df['l3_id'] = l3_id
# Шаг 1: Разворачиваем список в столбце trend
df_exploded = df.explode('trend')

# Шаг 2: Нормализуем словари в столбце trend
trend_df = pd.json_normalize(df_exploded['trend'])

# Шаг 3: Объединяем развернутые данные с исходным DataFrame
df_exploded = df_exploded.drop(columns=['trend']).reset_index(drop=True)
df_result = pd.concat([df_exploded, trend_df], axis=1)

In [146]:
df_result

,name,l3_id,date,ordersSum,orders
0,Книги > Политика и право,15470321782971428351,2025-08-23T00:00:00+00:00,288076,385
1,Книги > Политика и право,15470321782971428351,2025-08-24T00:00:00+00:00,307023,409
2,Книги > Политика и право,15470321782971428351,2025-08-25T00:00:00+00:00,282795,367
3,Книги > Политика и право,15470321782971428351,2025-08-26T00:00:00+00:00,328206,426
4,Книги > Политика и право,15470321782971428351,2025-08-27T00:00:00+00:00,392668,554
5,Книги > Политика и право,15470321782971428351,2025-08-28T00:00:00+00:00,457637,609
6,Книги > Политика и право,15470321782971428351,2025-08-29T00:00:00+00:00,372062,512
7,Книги > Политика и право,15470321782971428351,2025-08-30T00:00:00+00:00,445521,654
8,Книги > Политика и право,15470321782971428351,2025-08-31T00:00:00+00:00,512445,784
9,Книги > Политика и право,15470321782971428351,2025-09-01T00:00:00+00:00,424278,665


In [144]:
df_temp.columns

Index(['status', 'items', 'userBrands', 'recordsTotal', 'recordsFiltered',
       'draw', 'message', 'totals.brandId', 'totals.lastRemains',
       'totals.reviewsCount', 'totals.name', 'totals.isFavorite',
       'totals.orders', 'totals.ordersSum', 'totals.loosesPercent',
       'totals.avgSum', 'totals.loosesSum', 'totals.trend',
       'totals.productsCount', 'totals.newProductsCount',
       'totals.orderedCardsSum', 'totals.sharePercent', 'filters.orders.type',
       'filters.orders.min', 'filters.orders.max', 'filters.ordersSum.type',
       'filters.ordersSum.min', 'filters.ordersSum.max',
       'filters.loosesPercent.type', 'filters.loosesPercent.min',
       'filters.loosesPercent.max', 'filters.avgSum.type',
       'filters.avgSum.min', 'filters.avgSum.max', 'filters.loosesSum.type',
       'filters.loosesSum.min', 'filters.loosesSum.max', 'filters.name.type',
       'filters.name.value', 'filters.orderedCardsSum.type',
       'filters.orderedCardsSum.min', 'filters.ordere

In [147]:
data = get_info_30_days(yesterday_str, l3_id, today_str, session, cookies_dict)
df_temp = pd.json_normalize(data)
df_temp = df_temp[['name', 'trend']]
df_temp['l3_id'] = l3_id
# Шаг 1: Разворачиваем список в столбце trend
df_exploded = df_temp.explode('trend')

# Шаг 2: Нормализуем словари в столбце trend
trend_df = pd.json_normalize(df_exploded['trend'])

# Шаг 3: Объединяем развернутые данные с исходным DataFrame
df_exploded = df_exploded.drop(columns=['trend']).reset_index(drop=True)
df_temp = pd.concat([df_exploded, trend_df], axis=1)

https://eggheads.solutions/analytics/wbCategory/getBrandsList/15470321782971428351.json?query=%7B%22start%22%3A%200%2C%20%22length%22%3A%200%2C%20%22orderBy%22%3A%20%22ordersSum%22%2C%20%22orderDirection%22%3A%20%22desc%22%2C%20%22checkDate%22%3A%20%222025-09-21%22%2C%20%22periodDays%22%3A%20%2232%22%2C%20%22trendType%22%3A%20%22day%22%2C%20%22filters%22%3A%20%7B%22showFavoritesOnly%22%3A%20%7B%22value%22%3A%20false%7D%7D%7D&dns-cache=2025-09-22_09-1
No data for periodDays=32, retrying with periodDays=30
https://eggheads.solutions/analytics/wbCategory/getBrandsList/15470321782971428351.json?query=%7B%22start%22%3A%200%2C%20%22length%22%3A%200%2C%20%22orderBy%22%3A%20%22ordersSum%22%2C%20%22orderDirection%22%3A%20%22desc%22%2C%20%22checkDate%22%3A%20%222025-09-21%22%2C%20%22periodDays%22%3A%20%2230%22%2C%20%22trendType%22%3A%20%22day%22%2C%20%22filters%22%3A%20%7B%22showFavoritesOnly%22%3A%20%7B%22value%22%3A%20false%7D%7D%7D&dns-cache=2025-09-22_09-1


In [148]:
df_temp

,name,l3_id,date,ordersSum,orders
0,Книги > Политика и право,15470321782971428351,2025-08-23T00:00:00+00:00,288076,385
1,Книги > Политика и право,15470321782971428351,2025-08-24T00:00:00+00:00,307023,409
2,Книги > Политика и право,15470321782971428351,2025-08-25T00:00:00+00:00,282795,367
3,Книги > Политика и право,15470321782971428351,2025-08-26T00:00:00+00:00,328206,426
4,Книги > Политика и право,15470321782971428351,2025-08-27T00:00:00+00:00,392668,554
5,Книги > Политика и право,15470321782971428351,2025-08-28T00:00:00+00:00,457637,609
6,Книги > Политика и право,15470321782971428351,2025-08-29T00:00:00+00:00,372062,512
7,Книги > Политика и право,15470321782971428351,2025-08-30T00:00:00+00:00,445521,654
8,Книги > Политика и право,15470321782971428351,2025-08-31T00:00:00+00:00,512445,784
9,Книги > Политика и право,15470321782971428351,2025-09-01T00:00:00+00:00,424278,665


In [139]:
df_result

,name,l3_id,date,ordersSum,orders
0,Книги > Политика и право,15470321782971428351,2025-08-23T00:00:00+00:00,288076,385
1,Книги > Политика и право,15470321782971428351,2025-08-24T00:00:00+00:00,307023,409
2,Книги > Политика и право,15470321782971428351,2025-08-25T00:00:00+00:00,282795,367
3,Книги > Политика и право,15470321782971428351,2025-08-26T00:00:00+00:00,328206,426
4,Книги > Политика и право,15470321782971428351,2025-08-27T00:00:00+00:00,392668,554
5,Книги > Политика и право,15470321782971428351,2025-08-28T00:00:00+00:00,457637,609
6,Книги > Политика и право,15470321782971428351,2025-08-29T00:00:00+00:00,372062,512
7,Книги > Политика и право,15470321782971428351,2025-08-30T00:00:00+00:00,445521,654
8,Книги > Политика и право,15470321782971428351,2025-08-31T00:00:00+00:00,512445,784
9,Книги > Политика и право,15470321782971428351,2025-09-01T00:00:00+00:00,424278,665


In [134]:
df_result

,brandId,lastRemains,reviewsCount,name,isFavorite,orders,ordersSum,loosesPercent,avgSum,loosesSum,productsCount,newProductsCount,orderedCardsSum,sharePercent,date,ordersSum,orders
0,None,None,None,Книги > Политика и право,False,18699,11536991,9.82,617,1133109,None,None,0,0,2025-08-23T00:00:00+00:00,288076,385
1,None,None,None,Книги > Политика и право,False,18699,11536991,9.82,617,1133109,None,None,0,0,2025-08-24T00:00:00+00:00,307023,409
2,None,None,None,Книги > Политика и право,False,18699,11536991,9.82,617,1133109,None,None,0,0,2025-08-25T00:00:00+00:00,282795,367
3,None,None,None,Книги > Политика и право,False,18699,11536991,9.82,617,1133109,None,None,0,0,2025-08-26T00:00:00+00:00,328206,426
4,None,None,None,Книги > Политика и право,False,18699,11536991,9.82,617,1133109,None,None,0,0,2025-08-27T00:00:00+00:00,392668,554
5,None,None,None,Книги > Политика и право,False,18699,11536991,9.82,617,1133109,None,None,0,0,2025-08-28T00:00:00+00:00,457637,609
6,None,None,None,Книги > Политика и право,False,18699,11536991,9.82,617,1133109,None,None,0,0,2025-08-29T00:00:00+00:00,372062,512
7,None,None,None,Книги > Политика и право,False,18699,11536991,9.82,617,1133109,None,None,0,0,2025-08-30T00:00:00+00:00,445521,654
8,None,None,None,Книги > Политика и право,False,18699,11536991,9.82,617,1133109,None,None,0,0,2025-08-31T00:00:00+00:00,512445,784
9,None,None,None,Книги > Политика и право,False,18699,11536991,9.82,617,1133109,None,None,0,0,2025-09-01T00:00:00+00:00,424278,665


In [128]:
df = df.explode('trend')

In [130]:
df_trend = pd.json_normalize(df['trend'])

In [131]:
df_trend

,date,ordersSum,orders
0,2025-08-23T00:00:00+00:00,288076,385
1,2025-08-24T00:00:00+00:00,307023,409
2,2025-08-25T00:00:00+00:00,282795,367
3,2025-08-26T00:00:00+00:00,328206,426
4,2025-08-27T00:00:00+00:00,392668,554
5,2025-08-28T00:00:00+00:00,457637,609
6,2025-08-29T00:00:00+00:00,372062,512
7,2025-08-30T00:00:00+00:00,445521,654
8,2025-08-31T00:00:00+00:00,512445,784
9,2025-09-01T00:00:00+00:00,424278,665


In [132]:
df

,l3_id,l3_name,trend
0,15470321782971428351,Книги > Политика и право,"{'date': '2025-08-23T00:00:00+00:00', 'ordersS..."
0,15470321782971428351,Книги > Политика и право,"{'date': '2025-08-24T00:00:00+00:00', 'ordersS..."
0,15470321782971428351,Книги > Политика и право,"{'date': '2025-08-25T00:00:00+00:00', 'ordersS..."
0,15470321782971428351,Книги > Политика и право,"{'date': '2025-08-26T00:00:00+00:00', 'ordersS..."
0,15470321782971428351,Книги > Политика и право,"{'date': '2025-08-27T00:00:00+00:00', 'ordersS..."
0,15470321782971428351,Книги > Политика и право,"{'date': '2025-08-28T00:00:00+00:00', 'ordersS..."
0,15470321782971428351,Книги > Политика и право,"{'date': '2025-08-29T00:00:00+00:00', 'ordersS..."
0,15470321782971428351,Книги > Политика и право,"{'date': '2025-08-30T00:00:00+00:00', 'ordersS..."
0,15470321782971428351,Книги > Политика и право,"{'date': '2025-08-31T00:00:00+00:00', 'ordersS..."
0,15470321782971428351,Книги > Политика и право,"{'date': '2025-09-01T00:00:00+00:00', 'ordersS..."


In [22]:
data

{'status': 'ok',
 'items': [],
 'totals': None,
 'userBrands': [],
 'filters': {'orders': {'type': 'range', 'min': None, 'max': None},
  'ordersSum': {'type': 'range', 'min': None, 'max': None},
  'loosesPercent': {'type': 'range', 'min': None, 'max': None},
  'avgSum': {'type': 'range', 'min': None, 'max': None},
  'loosesSum': {'type': 'range', 'min': None, 'max': None},
  'name': {'type': 'text', 'value': None},
  'orderedCardsSum': {'type': 'range', 'min': None, 'max': None},
  'productsCount': {'type': 'range', 'min': None, 'max': None},
  'newProductsCount': {'type': 'range', 'min': None, 'max': None},
  'sharePercent': {'type': 'range', 'min': None, 'max': None},
  'lastRemains': {'type': 'range', 'min': None, 'max': None}},
 'request': {'filters': {},
  'checkDate': '2025-09-21T00:00:00+00:00',
  'periodDays': 30,
  'trendType': 'day',
  'draw': 0,
  'start': 0,
  'length': 0,
  'orderBy': 'ordersSum',
  'orderDirection': 'desc'},
 'recordsTotal': 0,
 'recordsFiltered': 0,
 'dr

In [ ]:
ur = endpoints.get_info_30_days(yesterday_str, l3_id, today_str, session, cookies_dict)

In [5]:
def get_wb_totals():
    # Параметры запроса
    category_id = "7079448242063961252"
    check_date = "2025-09-15"
    period_days = "30"

    # Формируем JSON query
    query_params = {
        "start": 0,
        "length": 0,
        "orderBy": "ordersSum",
        "orderDirection": "desc",
        "checkDate": check_date,
        "periodDays": period_days,
        "trendType": "day",
        "filters": {}
    }

    # Кодируем JSON для URL
    encoded_query = quote(json.dumps(query_params))

    # Формируем timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-0")

    # Собираем итоговый URL
    url = f"https://eggheads.solutions/analytics/wbCategory/getBrandsList/{category_id}.json?query={encoded_query}&dns-cache={timestamp}"

    return url

In [37]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-0")

In [38]:
timestamp

'2025-09-16_12-0'

In [ ]:
response = requests.get(get_wb_totals(), cookies=cookies_dict)
data = response.json()

In [10]:
data

{'status': 'ok',
 'items': [],
 'totals': None,
 'userBrands': [],
 'filters': {'orders': {'type': 'range', 'min': None, 'max': None},
  'ordersSum': {'type': 'range', 'min': None, 'max': None},
  'loosesPercent': {'type': 'range', 'min': None, 'max': None},
  'avgSum': {'type': 'range', 'min': None, 'max': None},
  'loosesSum': {'type': 'range', 'min': None, 'max': None},
  'name': {'type': 'text', 'value': None},
  'orderedCardsSum': {'type': 'range', 'min': None, 'max': None},
  'productsCount': {'type': 'range', 'min': None, 'max': None},
  'newProductsCount': {'type': 'range', 'min': None, 'max': None},
  'sharePercent': {'type': 'range', 'min': None, 'max': None},
  'lastRemains': {'type': 'range', 'min': None, 'max': None}},
 'request': {'filters': {},
  'checkDate': '2025-09-15T00:00:00+00:00',
  'periodDays': 30,
  'trendType': 'day',
  'draw': 0,
  'start': 0,
  'length': 0,
  'orderBy': 'ordersSum',
  'orderDirection': 'desc'},
 'recordsTotal': 0,
 'recordsFiltered': 0,
 'dr

In [57]:
data = respones.json()

In [ ]:
ur = get_info_30_days(yesterday_str, l3_id, today_str, session, cookies_dict)

In [8]:
ur = 'https://eggheads.solutions/analytics/wbCategory/getBrandsList/7079448242063961252.json?query=%7B%22start%22%3A%200%2C%20%22length%22%3A%200%2C%20%22orderBy%22%3A%20%22ordersSum%22%2C%20%22orderDirection%22%3A%20%22desc%22%2C%20%22checkDate%22%3A%20%222025-09-21%22%2C%20%22periodDays%22%3A%20%2230%22%2C%20%22trendType%22%3A%20%22day%22%2C%20%22filters%22%3A%20%7B%22showFavoritesOnly%22%3A%20%7B%22value%22%3A%20false%7D%7D%7D&dns-cache=2025-09-22_09-1'

In [9]:
response = requests.get(ur, cookies=cookies_dict)
data = response.json()

In [10]:
data

{'status': 'ok',
 'items': [],
 'totals': None,
 'userBrands': [],
 'filters': {'orders': {'type': 'range', 'min': None, 'max': None},
  'ordersSum': {'type': 'range', 'min': None, 'max': None},
  'loosesPercent': {'type': 'range', 'min': None, 'max': None},
  'avgSum': {'type': 'range', 'min': None, 'max': None},
  'loosesSum': {'type': 'range', 'min': None, 'max': None},
  'name': {'type': 'text', 'value': None},
  'orderedCardsSum': {'type': 'range', 'min': None, 'max': None},
  'productsCount': {'type': 'range', 'min': None, 'max': None},
  'newProductsCount': {'type': 'range', 'min': None, 'max': None},
  'sharePercent': {'type': 'range', 'min': None, 'max': None},
  'lastRemains': {'type': 'range', 'min': None, 'max': None}},
 'request': {'filters': {'showFavoritesOnly': {'type': 'boolean',
    'value': False}},
  'checkDate': '2025-09-21T00:00:00+00:00',
  'periodDays': 30,
  'trendType': 'day',
  'draw': 0,
  'start': 0,
  'length': 0,
  'orderBy': 'ordersSum',
  'orderDirectio

In [14]:
ur

'https://eggheads.solutions/analytics/wbCategory/getBrandsList/7079448242063961252.json?query=%7B%22start%22%3A%200%2C%20%22length%22%3A%200%2C%20%22orderBy%22%3A%20%22ordersSum%22%2C%20%22orderDirection%22%3A%20%22desc%22%2C%20%22checkDate%22%3A%20%222025-09-21%22%2C%20%22periodDays%22%3A%20%2230%22%2C%20%22trendType%22%3A%20%22day%22%2C%20%22filters%22%3A%20%7B%7D%7D&dns-cache=2025-09-22_12-0'

In [13]:
data

{'status': 'ok',
 'items': [],
 'totals': None,
 'userBrands': [],
 'filters': {'orders': {'type': 'range', 'min': None, 'max': None},
  'ordersSum': {'type': 'range', 'min': None, 'max': None},
  'loosesPercent': {'type': 'range', 'min': None, 'max': None},
  'avgSum': {'type': 'range', 'min': None, 'max': None},
  'loosesSum': {'type': 'range', 'min': None, 'max': None},
  'name': {'type': 'text', 'value': None},
  'orderedCardsSum': {'type': 'range', 'min': None, 'max': None},
  'productsCount': {'type': 'range', 'min': None, 'max': None},
  'newProductsCount': {'type': 'range', 'min': None, 'max': None},
  'sharePercent': {'type': 'range', 'min': None, 'max': None},
  'lastRemains': {'type': 'range', 'min': None, 'max': None}},
 'request': {'filters': {},
  'checkDate': '2025-09-21T00:00:00+00:00',
  'periodDays': 30,
  'trendType': 'day',
  'draw': 0,
  'start': 0,
  'length': 0,
  'orderBy': 'ordersSum',
  'orderDirection': 'desc'},
 'recordsTotal': 0,
 'recordsFiltered': 0,
 'dr

'https://eggheads.solutions/analytics/wbCategory/getBrandsList/7079448242063961252.json?query=%7B%22start%22%3A%200%2C%20%22length%22%3A%200%2C%20%22orderBy%22%3A%20%22ordersSum%22%2C%20%22orderDirection%22%3A%20%22desc%22%2C%20%22checkDate%22%3A%20%222025-09-15%22%2C%20%22periodDays%22%3A%20%2230%22%2C%20%22trendType%22%3A%20%22day%22%2C%20%22filters%22%3A%20%7B%7D%7D&dns-cache=2025-09-16_12-0'

In [ ]:
'https://eggheads.solutions/analytics/wbCategory/getBrandsList/7079448242063961252.json?query=%7B%22start%22%3A%200%2C%20%22length%22%3A%200%2C%20%22orderBy%22%3A%20%22ordersSum%22%2C%20%22orderDirection%22%3A%20%22desc%22%2C%20%22checkDate%22%3A%20%222025-09-15%22%2C%20%22periodDays%22%3A%20%2230%22%2C%20%22trendType%22%3A%20%22day%22%2C%20%22filters%22%3A%20%7B%7D%7D&dns-cache=2025-09-16_12-0'

In [ ]:
'https://eggheads.solutions/analytics/wbCategory/getBrandsList/7079448242063961252.json?query=%7B%22start%22%3A%200%2C%20%22length%22%3A%200%2C%20%22orderBy%22%3A%20%22ordersSum%22%2C%20%22orderDirection%22%3A%20%22desc%22%2C%20%22checkDate%22%3A%20%222025-09-15%22%2C%20%22periodDays%22%3A%20%2230%22%2C%20%22trendType%22%3A%20%22day%22%2C%20%22filters%22%3A%20%7B%7D%7D&dns-cache=2025-09-16_12-0'

In [66]:
url

'https://eggheads.solutions/analytics/wbCategory/getBrandsList/7079448242063961252.json?query=%7B%22start%22%3A%200%2C%20%22length%22%3A%200%2C%20%22orderBy%22%3A%20%22ordersSum%22%2C%20%22orderDirection%22%3A%20%22desc%22%2C%20%22checkDate%22%3A%20%222025-09-15%22%2C%20%22periodDays%22%3A%20%2230%22%2C%20%22trendType%22%3A%20%22day%22%2C%20%22filters%22%3A%20%7B%7D%7D&dns-cache=2025-09-16_12-0'

In [58]:
data

{'status': 'ok',
 'items': [],
 'totals': {'brandId': None,
  'lastRemains': None,
  'reviewsCount': None,
  'name': 'Автотовары > OFFroad',
  'isFavorite': False,
  'orders': 5235,
  'ordersSum': 14220540,
  'loosesPercent': 60.67,
  'avgSum': 2716,
  'loosesSum': 8627879,
  'trend': [{'date': '2025-08-12T00:00:00+00:00',
    'ordersSum': 335784,
    'orders': 144},
   {'date': '2025-08-13T00:00:00+00:00', 'ordersSum': 462664, 'orders': 163},
   {'date': '2025-08-14T00:00:00+00:00', 'ordersSum': 421512, 'orders': 169},
   {'date': '2025-08-15T00:00:00+00:00', 'ordersSum': 409649, 'orders': 130},
   {'date': '2025-08-16T00:00:00+00:00', 'ordersSum': 417105, 'orders': 150},
   {'date': '2025-08-17T00:00:00+00:00', 'ordersSum': 410512, 'orders': 131},
   {'date': '2025-08-18T00:00:00+00:00', 'ordersSum': 400669, 'orders': 151},
   {'date': '2025-08-19T00:00:00+00:00', 'ordersSum': 437644, 'orders': 142},
   {'date': '2025-08-20T00:00:00+00:00', 'ordersSum': 335523, 'orders': 119},
   {'d

In [ ]:
data =

In [11]:
data = response.json()

In [12]:
data

{'status': 'ok',
 'items': [],
 'totals': {'brandId': None,
  'lastRemains': None,
  'reviewsCount': None,
  'name': 'Автотовары > OFFroad > Антипробуксовочное приспособление',
  'isFavorite': False,
  'orders': 755,
  'ordersSum': 1523102,
  'loosesPercent': 39.77,
  'avgSum': 2017,
  'loosesSum': 605759,
  'trend': [{'date': '2025-08-12T00:00:00+00:00',
    'ordersSum': 25652,
    'orders': 14},
   {'date': '2025-08-13T00:00:00+00:00', 'ordersSum': 25678, 'orders': 17},
   {'date': '2025-08-14T00:00:00+00:00', 'ordersSum': 39771, 'orders': 22},
   {'date': '2025-08-15T00:00:00+00:00', 'ordersSum': 19624, 'orders': 13},
   {'date': '2025-08-16T00:00:00+00:00', 'ordersSum': 43742, 'orders': 27},
   {'date': '2025-08-17T00:00:00+00:00', 'ordersSum': 28443, 'orders': 14},
   {'date': '2025-08-18T00:00:00+00:00', 'ordersSum': 31479, 'orders': 17},
   {'date': '2025-08-19T00:00:00+00:00', 'ordersSum': 32790, 'orders': 15},
   {'date': '2025-08-20T00:00:00+00:00', 'ordersSum': 21538, 'order

,ordersSum,orders,dt
0,25652,14,2025-08-12 00:00:00+00:00
1,25678,17,2025-08-13 00:00:00+00:00
2,39771,22,2025-08-14 00:00:00+00:00
3,19624,13,2025-08-15 00:00:00+00:00
4,43742,27,2025-08-16 00:00:00+00:00
5,28443,14,2025-08-17 00:00:00+00:00
6,31479,17,2025-08-18 00:00:00+00:00
7,32790,15,2025-08-19 00:00:00+00:00
8,21538,13,2025-08-20 00:00:00+00:00
9,27910,12,2025-08-21 00:00:00+00:00


In [22]:
df

,date,ordersSum,orders,dt
0,2025-08-12T00:00:00+00:00,25652,14,2025-08-12 00:00:00+00:00
1,2025-08-13T00:00:00+00:00,25678,17,2025-08-13 00:00:00+00:00
2,2025-08-14T00:00:00+00:00,39771,22,2025-08-14 00:00:00+00:00
3,2025-08-15T00:00:00+00:00,19624,13,2025-08-15 00:00:00+00:00
4,2025-08-16T00:00:00+00:00,43742,27,2025-08-16 00:00:00+00:00
5,2025-08-17T00:00:00+00:00,28443,14,2025-08-17 00:00:00+00:00
6,2025-08-18T00:00:00+00:00,31479,17,2025-08-18 00:00:00+00:00
7,2025-08-19T00:00:00+00:00,32790,15,2025-08-19 00:00:00+00:00
8,2025-08-20T00:00:00+00:00,21538,13,2025-08-20 00:00:00+00:00
9,2025-08-21T00:00:00+00:00,27910,12,2025-08-21 00:00:00+00:00


In [16]:
df = df.explode('trend')

In [ ]:
df = df[['']]

,brandId,lastRemains,reviewsCount,name,isFavorite,orders,ordersSum,loosesPercent,avgSum,loosesSum,trend,productsCount,newProductsCount,orderedCardsSum,sharePercent
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-12T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-13T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-14T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-15T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-16T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-17T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-18T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-19T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-20T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-21T00:00:00+00:00', 'ordersS...",None,None,0,0


In [3]:
pd.set_option('display.max_rows', None)

In [4]:
df = pd.read_csv('l1_l2_data.csv')

In [31]:
# Заменяем строки с кавычками на NaN
df['l2_id'] = df['l2_id'].replace('', pd.NA, regex=False)

# Заполняем NaN значениями из l1_id
df['l2_id'] = df['l2_id'].fillna(df['l1_id'])